In [1]:
import logging
import requests
import time
import pandas as pd
import pyodbc
import urllib.request
import urllib.parse
import datetime
import json
from io import BytesIO
from io import StringIO
from dateutil.relativedelta import relativedelta
from azure.storage.blob import BlockBlobService, PublicAccess

In [2]:
date_format = '%m-%d-%Y'

In [3]:
headers = ['TickerSymbol', 'Date', 'Open', 'High', 'Low', 'Close', 'OI', 'Volume']

In [4]:
expiration_months_market_commodities = {
        'C': [['H', 'K', 'N', 'U', 'Z'], 'CBOT'],
        'W': [['H', 'K', 'N', 'U', 'Z'], 'CBOT'],
        'KW': [['H', 'K', 'N', 'U', 'Z'], 'KCBT'],
        'MW': [['H', 'K', 'N', 'U', 'Z'], ''],
        'S': [['F', 'H', 'K', 'N', 'Q', 'U', 'X'], 'CBOT'],
        'SM': [['F', 'H', 'K', 'N', 'Q', 'U', 'V', 'Z'], 'CBOT'],
        'BO': [['F', 'H', 'K', 'N', 'Q', 'U', 'V', 'Z'], 'CBOT']
    }

In [5]:
base_url_prophetex = 'http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/'
history_method = 'GetDailyHistory'

In [6]:
def get_last_record_date(sql):
    server = 'grainpredictive.database.windows.net'
    database = 'gpi'
    username = 'gpi'
    password = 'Cmi@2019$A'
    driver= '{ODBC Driver 17 for SQL Server}'
    cnxn = pyodbc.connect('DRIVER='+driver+';SERVER='+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password)
    cursor = cnxn.cursor()
    cursor.execute(sql)
    dates = []
    for row in cursor:
        date_row = []
        commodity = row[0]
        date = row[1]
        date_row.append(commodity)
        date_row.append(date)
        dates.append(date_row)
    return dates

In [7]:
#NEW.
def get_futures_prices_historical_expirations(symbol, year, code):
    #Request: @C2000U [CORN, U: September 2000]
    futures_data = pd.DataFrame(columns=['TickerSymbol', 'Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'OI', 'ExpirationSymbol'])
    time.sleep(2)
    url = base_url_prophetex + history_method
    params = urllib.parse.urlencode(
        {
            'UserID':'ws@mfgrains.com',
            'Password': 'Kr5o8N',
            'Symbol': '@'+symbol+str(year)+code,
            'Limit': 8000,
            'Format': 'CSV'
        }
    )
    req = urllib.request.urlopen(url + '?%s' % params)
    print(req.geturl())
    content = req.read()        
    df = pd.read_csv(StringIO(content.decode()), header=0)
    df['ExpirationSymbol'] ='@'+symbol+str(year)+code
    futures_data = futures_data.append(df)
    return futures_data

In [8]:
def upload_azure(values, file_name):
    # Create the BlockBlockService that is used to call the Blob service for the storage account
    block_blob_service = BlockBlobService(account_name='gpistore', account_key='zfKM5R0PuPwR0F+pPsgs5BW/AQjAxv5fwKojoP2W38II++qfT6e+axFrRAcTOmKi/8U0tyJbrB2A3XCd7W7o6A==')

    # Create a container called 'quickstartblobs'.
    container_name ='gpistore'
    block_blob_service.create_container(container_name)

    # Set the permission so the blobs are public.
    block_blob_service.set_container_acl(container_name, public_access=PublicAccess.Container)    

    #Extraccion de datos
    name = file_name+'.csv'
    values_csv = values.to_csv(header=True,index=False,encoding='utf-8-sig')
    block_blob_service.create_blob_from_text(container_name,name,values_csv)
    return f'Extraccion de ProphetX  de {file_name} exitosa.'


In [9]:
def COMMODITIES_PRICE(type):
    #Cambiar el Date a Year.
    if type == 'CORN':
        sql_prices = "SELECT * FROM (SELECT DISTINCT commodity, YEAR(fecha) as Year FROM(SELECT [Date] fecha,	REPLACE(LEFT(TickerSymbol, LEN(TickerSymbol)-2), '@', '') commodity FROM [ST_PROPHETX].[COMMODITIES_PRICE_HISTORY_CC] ) t) as t WHERE t.commodity IN ('C') ORDER BY [Year]"
    elif type == 'WHEAT':
        sql_prices = "SELECT * FROM (SELECT DISTINCT commodity, YEAR(fecha) as Year FROM(SELECT [Date] fecha,	REPLACE(LEFT(TickerSymbol, LEN(TickerSymbol)-2), '@', '') commodity FROM [ST_PROPHETX].[COMMODITIES_PRICE_HISTORY_CC] ) t) as t WHERE t.commodity IN ('W','KW','MW') ORDER BY [Year]"
    elif type == 'SOYBEAN':
        sql_prices = "SELECT * FROM (SELECT DISTINCT commodity, YEAR(fecha) as Year FROM(SELECT [Date] fecha,	REPLACE(LEFT(TickerSymbol, LEN(TickerSymbol)-2), '@', '') commodity FROM [ST_PROPHETX].[COMMODITIES_PRICE_HISTORY_CC] ) t) as t WHERE t.commodity IN ('S','SM','BO') ORDER BY [Year]"

    records = get_last_record_date(sql_prices)
    data = pd.DataFrame(columns=headers)
    for i in range(len(records)):
        commodity = records[i][0]
        #Ej. C
        print('Running Commodity: ' + commodity)
        year = records[i][1]
        #2000-2020
        print('Running Year: '+ str(year))
        exp_month_cd = expiration_months_market_commodities.get(commodity)
        #Ej. 'H', 'K', 'N', 'U', 'Z'
        for exp in exp_month_cd[0]:
            print('Running Exp Month: ' + exp)
            data_next_n_expirations = get_futures_prices_historical_expirations(commodity, year, exp)
            data = data.append(data_next_n_expirations[headers])
    data['actualizacion'] = datetime.datetime.now()
    
    if data.empty:
        r = '{"Result":"False"}'
    else:
        r = '{"Result":"True"}'
        data = data.replace('---','0')     
    data.to_csv('COMMODITIES_PRICE.csv',index=False)
    return r

In [18]:
#CORN.
COMMODITIES_PRICE('CORN')

Running Commodity: C
Running Year: 2001
Running Exp Month: H
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40C2001H&Limit=8000&Format=CSV
Running Exp Month: K
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40C2001K&Limit=8000&Format=CSV
Running Exp Month: N
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40C2001N&Limit=8000&Format=CSV
Running Exp Month: U
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40C2001U&Limit=8000&Format=CSV
Running Exp Month: Z
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40C2001Z&Limit=8000&Format=CSV
Running Commodity: C
Running Year: 2002
Running Exp Month: H
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgr

http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40C2010U&Limit=8000&Format=CSV
Running Exp Month: Z
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40C2010Z&Limit=8000&Format=CSV
Running Commodity: C
Running Year: 2011
Running Exp Month: H
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40C2011H&Limit=8000&Format=CSV
Running Exp Month: K
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40C2011K&Limit=8000&Format=CSV
Running Exp Month: N
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40C2011N&Limit=8000&Format=CSV
Running Exp Month: U
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40C2011U&Limit=8000&Format=C

http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40C2020K&Limit=8000&Format=CSV
Running Exp Month: N
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40C2020N&Limit=8000&Format=CSV
Running Exp Month: U
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40C2020U&Limit=8000&Format=CSV
Running Exp Month: Z
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40C2020Z&Limit=8000&Format=CSV


'{"Result":"True"}'

In [10]:
#WHEAT.
COMMODITIES_PRICE('WHEAT')

Running Commodity: KW
Running Year: 2001
Running Exp Month: H
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40KW2001H&Limit=8000&Format=CSV
Running Exp Month: K
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40KW2001K&Limit=8000&Format=CSV
Running Exp Month: N
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40KW2001N&Limit=8000&Format=CSV
Running Exp Month: U
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40KW2001U&Limit=8000&Format=CSV
Running Exp Month: Z
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40KW2001Z&Limit=8000&Format=CSV
Running Commodity: MW
Running Year: 2001
Running Exp Month: H
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws

http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40KW2004U&Limit=8000&Format=CSV
Running Exp Month: Z
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40KW2004Z&Limit=8000&Format=CSV
Running Commodity: MW
Running Year: 2004
Running Exp Month: H
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40MW2004H&Limit=8000&Format=CSV
Running Exp Month: K
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40MW2004K&Limit=8000&Format=CSV
Running Exp Month: N
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40MW2004N&Limit=8000&Format=CSV
Running Exp Month: U
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40MW2004U&Limit=8000&F

http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40MW2007K&Limit=8000&Format=CSV
Running Exp Month: N
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40MW2007N&Limit=8000&Format=CSV
Running Exp Month: U
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40MW2007U&Limit=8000&Format=CSV
Running Exp Month: Z
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40MW2007Z&Limit=8000&Format=CSV
Running Commodity: W
Running Year: 2007
Running Exp Month: H
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40W2007H&Limit=8000&Format=CSV
Running Exp Month: K
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40W2007K&Limit=8000&Form

http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40MW2010Z&Limit=8000&Format=CSV
Running Commodity: W
Running Year: 2010
Running Exp Month: H
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40W2010H&Limit=8000&Format=CSV
Running Exp Month: K
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40W2010K&Limit=8000&Format=CSV
Running Exp Month: N
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40W2010N&Limit=8000&Format=CSV
Running Exp Month: U
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40W2010U&Limit=8000&Format=CSV
Running Exp Month: Z
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40W2010Z&Limit=8000&Format=

Running Exp Month: N
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40W2013N&Limit=8000&Format=CSV
Running Exp Month: U
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40W2013U&Limit=8000&Format=CSV
Running Exp Month: Z
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40W2013Z&Limit=8000&Format=CSV
Running Commodity: KW
Running Year: 2014
Running Exp Month: H
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40KW2014H&Limit=8000&Format=CSV
Running Exp Month: K
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40KW2014K&Limit=8000&Format=CSV
Running Exp Month: N
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40KW

Running Commodity: KW
Running Year: 2017
Running Exp Month: H
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40KW2017H&Limit=8000&Format=CSV
Running Exp Month: K
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40KW2017K&Limit=8000&Format=CSV
Running Exp Month: N
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40KW2017N&Limit=8000&Format=CSV
Running Exp Month: U
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40KW2017U&Limit=8000&Format=CSV
Running Exp Month: Z
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40KW2017Z&Limit=8000&Format=CSV
Running Commodity: MW
Running Year: 2017
Running Exp Month: H
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws

Running Exp Month: U
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40KW2020U&Limit=8000&Format=CSV
Running Exp Month: Z
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40KW2020Z&Limit=8000&Format=CSV
Running Commodity: MW
Running Year: 2020
Running Exp Month: H
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40MW2020H&Limit=8000&Format=CSV
Running Exp Month: K
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40MW2020K&Limit=8000&Format=CSV
Running Exp Month: N
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40MW2020N&Limit=8000&Format=CSV
Running Exp Month: U
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%4

'{"Result":"True"}'

In [10]:
#SOYBEAN.
COMMODITIES_PRICE('SOYBEAN')

Running Commodity: BO
Running Year: 2001
Running Exp Month: F
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40BO2001F&Limit=8000&Format=CSV
Running Exp Month: H
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40BO2001H&Limit=8000&Format=CSV
Running Exp Month: K
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40BO2001K&Limit=8000&Format=CSV
Running Exp Month: N
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40BO2001N&Limit=8000&Format=CSV
Running Exp Month: Q
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40BO2001Q&Limit=8000&Format=CSV
Running Exp Month: U
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%4

http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40BO2003K&Limit=8000&Format=CSV
Running Exp Month: N
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40BO2003N&Limit=8000&Format=CSV
Running Exp Month: Q
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40BO2003Q&Limit=8000&Format=CSV
Running Exp Month: U
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40BO2003U&Limit=8000&Format=CSV
Running Exp Month: V
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40BO2003V&Limit=8000&Format=CSV
Running Exp Month: Z
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40BO2003Z&Limit=8000&Format=CSV
Running Commodity: S
Running Ye

http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40BO2005U&Limit=8000&Format=CSV
Running Exp Month: V
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40BO2005V&Limit=8000&Format=CSV
Running Exp Month: Z
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40BO2005Z&Limit=8000&Format=CSV
Running Commodity: S
Running Year: 2005
Running Exp Month: F
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40S2005F&Limit=8000&Format=CSV
Running Exp Month: H
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40S2005H&Limit=8000&Format=CSV
Running Exp Month: K
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40S2005K&Limit=8000&Forma

http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40S2007F&Limit=8000&Format=CSV
Running Exp Month: H
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40S2007H&Limit=8000&Format=CSV
Running Exp Month: K
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40S2007K&Limit=8000&Format=CSV
Running Exp Month: N
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40S2007N&Limit=8000&Format=CSV
Running Exp Month: Q
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40S2007Q&Limit=8000&Format=CSV
Running Exp Month: U
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40S2007U&Limit=8000&Format=CSV
Running Exp Month: X
http://pxweb.dtn

http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40S2009N&Limit=8000&Format=CSV
Running Exp Month: Q
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40S2009Q&Limit=8000&Format=CSV
Running Exp Month: U
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40S2009U&Limit=8000&Format=CSV
Running Exp Month: X
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40S2009X&Limit=8000&Format=CSV
Running Commodity: SM
Running Year: 2009
Running Exp Month: F
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40SM2009F&Limit=8000&Format=CSV
Running Exp Month: H
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40SM2009H&Limit=8000&Forma

...

http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40SM2013K&Limit=8000&Format=CSV
Running Exp Month: N
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40SM2013N&Limit=8000&Format=CSV
Running Exp Month: Q
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40SM2013Q&Limit=8000&Format=CSV
Running Exp Month: U
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40SM2013U&Limit=8000&Format=CSV
Running Exp Month: V
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40SM2013V&Limit=8000&Format=CSV
Running Exp Month: Z
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40SM2013Z&Limit=8000&Format=CSV
Running Commodity: BO
Running Y

Running Exp Month: U
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40SM2015U&Limit=8000&Format=CSV
Running Exp Month: V
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40SM2015V&Limit=8000&Format=CSV
Running Exp Month: Z
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40SM2015Z&Limit=8000&Format=CSV
Running Commodity: BO
Running Year: 2016
Running Exp Month: F
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40BO2016F&Limit=8000&Format=CSV
Running Exp Month: H
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40BO2016H&Limit=8000&Format=CSV
Running Exp Month: K
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%4

Running Commodity: BO
Running Year: 2018
Running Exp Month: F
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40BO2018F&Limit=8000&Format=CSV
Running Exp Month: H
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40BO2018H&Limit=8000&Format=CSV
Running Exp Month: K
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40BO2018K&Limit=8000&Format=CSV
Running Exp Month: N
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40BO2018N&Limit=8000&Format=CSV
Running Exp Month: Q
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40BO2018Q&Limit=8000&Format=CSV
Running Exp Month: U
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%4

Running Exp Month: K
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40BO2020K&Limit=8000&Format=CSV
Running Exp Month: N
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40BO2020N&Limit=8000&Format=CSV
Running Exp Month: Q
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40BO2020Q&Limit=8000&Format=CSV
Running Exp Month: U
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40BO2020U&Limit=8000&Format=CSV
Running Exp Month: V
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40BO2020V&Limit=8000&Format=CSV
Running Exp Month: Z
http://pxweb.dtn.com/PXWebSvc/PXServiceWeb.svc/GetDailyHistory?UserID=ws%40mfgrains.com&Password=Kr5o8N&Symbol=%40BO2020Z&Limit=8000&Format=CSV
Running Co

'{"Result":"True"}'